# Comparison of Hessian-vector product

Often times we need Hessian-vector products in numerical computation / optimization. A nice family of examples of using Hessian-vector products is Krylov subspace based iterative algorithms such as Conjugate Gradient, Lanczos method, and Power method. In deep learning literature, Krylov subspace is often used in the context of 2nd-order optimization. Please refer to [Deep learning via Hessian-free optimization](http://www.cs.toronto.edu/~jmartens/docs/Deep_HessianFree.pdf), [Krylov Subspace Descent for Deep Learning](http://www1.icsi.berkeley.edu/~vinyals/Files/vinyals_aistats12.pdf), and [Identifying and attacking the saddle point problem in
high-dimensional non-convex optimization](https://arxiv.org/pdf/1406.2572v1.pdf) for more detail. 

There are three methods of computing Hessian-vector products that we can try in TensorFlow.

1. Compute Hessian directly and multiply the result with a vector.
2. Use finite differences to get Hessian and multiply the result with a vector. (used in Marten's paper)
3. Use Pearlmutter trick. (will explain later)

In this post, I will compare these methods in terms of numerical stability and computational speed. 
I'm mainly interested in how closely well the finite differences would work when the parameter space gets big / when the condition number of Hessian becomes large, compared to Pearlmutter trick. 

The Method 1 is just for illustrative purpose. In practice, one should not compute Hessian directly because it's too much computation. 

The reason why I included method 2 is just for reference. I've tried to compute Hessian-vector product once in Torch7, which doesn't have an easy way to do the method 3. For TensorFlow / Theano users, Method 3 should suffice.   

I made a TensorFlow function to compute Hessians for multilayer perceptron before, so I'll reuse it to try out the method 1.

In [2]:
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import math
import numpy as np
import tensorflow as tf
import time

#from datasets import dataset_utils

# Main slim library
slim = tf.contrib.slim

# Speed comparison

## Method 1

In [ ]:
def method1(v):
    H_1 = getHessian()
    return np.dot(H_1, v)

## Method 2

In [ ]:
def method2(v):
    with tf.Graph().as_default():
        inputs = 
        targets = 
        # Compute finite differences of gradients
        output, end_points = build_MLP(inputs, n_hidden=10)
        # loss = slim.losses.sum_of_squares(predictions, targets) # Slim way
        # The total loss is the uers's loss plus any regularization losses.
        # total_loss = slim.losses.get_total_loss() # Slim way
        loss = tf.reduce_mean(tf.square(output - targets))
        grad = tf.gradients(loss) # make sure that this returns a vector, not a list of gradient and something else.
        loss_eps = tf.add(loss, eps)
        grad_eps = tf.gradients(loss_eps) # make sure that this returns a vector, not a list of gradient and something else.
        Hv = tf.div(grad - grad_eps, eps)
        with tf.Session() as sess:
            sess.run(tf.initialize_all_variables())
            print(sess.run(Hv))

In [6]:
def build_MLP(inputs, n_hidden=5):
    end_points = {}
    with slim.arg_scope([slim.fully_connected], 
                        activation_fn=tf.nn.relu,
                        weights_regularizer=slim.l2_regularizer(0.01)):
        net = slim.fully_connected(inputs, n_hidden, scope="fc1")
        end_points["fc1"] = net 
        output = slim.fully_connected(net, n_output, activation_fn=None, scope="output")
        end_points["output"] = output
        
        return output, end_points

## Method 3 : Pearlmutter trick

Pearlmutter trick is a way to compute $Hv$ in an efficient way using backpropagation. The basic idea is to take the gradient of ----
$$
\nabla_{\theta}(g^T v) = H^T v  = Hv 
$$
, where g is a gradient of the objective funtion with respect to parameters. 

In [8]:
def method3(v):
    with tf.Graph().as_default():
        inputs = 
        targets = 
        output, end_points = build_MLP(inputs, n_hidden=10)
        loss = tf.reduce_mean(tf.square(output - targets))
        grad = tf.gradients(loss) # make sure that this returns a vector, not a list of gradient and something else.
        Hv = tf.gradients(tf.mul(grad, v))
        with tf.Session() as sess:
            sess.run(tf.initialize_all_variables())
            print(sess.run(Hv))

SyntaxError: invalid syntax (<ipython-input-8-5bc8f0f6582b>, line 3)

# Condition number 

Condition number, as a concept, captures sensitivity of a system, which has inputs and outputs. It measures how outputs will change with respect to change in the inputs. Many problems **(examples?)** can be reduced to finding $x$ s.t. $Ax = b$. In this case, $b$ is the input and $x$ is the solution we want, and $A$ represents the structure of the problem / system you are interested in. There are a number of types of condition number depending on how to measure the "sensitivity" of the system. Let us focus on the case where $X$ is a symmetric matrix because then the definition of condition number is simple: 

---

Condition number of a symmetric matrix X is defined as 
$$ 
\frac{|\lambda_{max}|}{|\lambda_{min}|} 
$$

---

where $\lambda_{max / min}$ is the maximum / minimum eigenvalue of the matrix $X$. For those who are interested to know why it is so, please refer to my other blog post on [induced matrix norm](). 

Let us use a simple quadratic function for our objective funtion because it's easier to control the condition number of the Hessian; we can control the condition number of a matrix through SVD by modifying the resulting eigenvalues of the diagonal matrix returned by SVD.  

For any quadratic function s.t. $f(x) = \frac{1}{2} x^T A x + b^T x + c$, the Hessian of f(x) is A when A is symmetric. So we first create a random symmetric matrix like this:

In [7]:
def createRandomSymmMatrix(n_dim, condition_num): 
    random_mat = np.random.random(n_dim*n_dim).reshape(n_dim, n_dim) 
    random_mat = np.dot(np.transpose(random_mat), random_mat) # For any X, X^T X is always symmetric. 
    U,S,Vt = np.linalg.svd(random_mat) # Do svd to get diagonal elements. Note that S is a vector. 
    S = np.repeat(1.0, n_dim) # Replace the diagonal elements with a vector of 1s. 
    S[-1] = 1.0/condition_num # Let the last element of S be very small. This will determine condition number. 
    return np.dot(np.dot(U, np.diag(S)), Vt)

In [14]:
createRandomSymmMatrix(2, 2) 

array([[ 0.56449878,  0.16759862],
       [ 0.16759862,  0.93550122]])

## Method 2: Finite differences  

## Method 3: Pearlmutter Trick

In [13]:
def method3_c(v, n_dim, condition_num):
    with tf.Graph().as_default():
        inputs = createInputs(n_dim, condition_num)
        output, matrix = build_quadratic(inputs)
        loss = tf.reduce_sum(output - 0)
        tvars = tf.trainable_variables()
        grad = tf.gradients(loss, tvars)[0]      
        Hv = tf.gradients(tf.matmul(tf.transpose(grad), v), tvars)
        with tf.Session() as sess:
            sess.run(tf.initialize_all_variables())
            print(sess.run(grad).shape)
            print(sess.run(tf.matmul(tf.transpose(grad), v)))
            print(sess.run(Hv))
    return matrix

In [12]:
n_dim = 5
n_cond_num = 2
vec = np.repeat(1.0, n_dim).reshape([n_dim, 1])
matrix = method3_c(vec, n_dim, n_cond_num) 
print(np.dot(np.transpose(matrix), vec))

[[ 0.6793354   0.16358218 -0.0996524   0.07539995  0.12301069]
 [ 0.16358218  0.91655103  0.05083616 -0.03846414 -0.06275204]
 [-0.0996524   0.05083616  0.96903119  0.02343192  0.03822783]
 [ 0.07539995 -0.03846414  0.02343192  0.98227072 -0.0289243 ]
 [ 0.12301069 -0.06275204  0.03822783 -0.0289243   0.95281166]]
(5, 1)
[[ 7.53296693]]
[array([[ 0.94167582],
       [ 1.02975319],
       [ 0.98187469],
       [ 1.01371414],
       [ 1.02237384]])]
[[ 0.94167582]
 [ 1.02975319]
 [ 0.98187469]
 [ 1.01371414]
 [ 1.02237384]]


In [3]:
def build_quadratic(inputs):
    A = inputs["A"]
    print(A)
    b = inputs["b"]
    c = inputs["c"]
    n_dim, _ = np.shape(A)
    
    x = tf.Variable(np.float64(np.repeat(1.0,n_dim).reshape(n_dim,1)))
    
    # Construct the computational graph for quadratic function: f(x) = 1/2 * x^t A x + b^t x + c
    output = 0.5 * tf.matmul(tf.matmul(tf.transpose(x), A), x) + tf.matmul(tf.transpose(b), x) + c
    return output, A

In [4]:
def createInputs(n_dim, condition_num):
    inputs = dict() 
    inputs["A"] = createRandomSymmMatrix(n_dim, condition_num)
    inputs["b"] = np.random.random(n_dim).reshape(n_dim, 1)
    inputs["c"] = np.random.random(1)
    return inputs

In [41]:
np.linalg.cond(createMatrix(3,10)) 

10.000000000000005

Looks good. 

## Reference

Next, let's compare the result of the above function with a method using finite differences. 

(By the way, there is a way to compute Hessian-vector product using only gradients, which is called Pearlmutter trick in neural network literature. Please refer to https://cswhjiang.github.io/2015/10/13/Roperator/ for more details.)